# Conversion  of CSV to SQL Using Python

In [ ]:
#Import all the required libraries

import glob
import os
import pandas as pd
import pyodbc
import csv
import time
import schedule
import urllib
import sqlalchemy


In [ ]:
def csv_to_sql():
    #Server Credentials
    server = '10.7.8.4' 
    database = 'IOT_REST' 
    username = 'iotuser' 
    password = 'iot@bfl1234' 
    Table_Name='CSVSQL2_TEST'
    
    #Creating a connection object and connect it to the database.

    params = urllib.parse.quote_plus('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    engine=sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
    engine.connect()
    
    
    # Logic to find the latest created file.
    list_of_files=glob.glob('D:\\datafrompendrive\\*')
    latest_file= max(list_of_files,key=os.path.getctime)
    print( " Newly Created File is : ",latest_file)
    df=pd.read_csv(latest_file,skiprows=31)
    
    
    #Logic to write the data to the SQL Server
    data=df[['Inspection Date Time','Job Serial Number','Layout','RUNOUT','DIA','Shift']]
    data.rename(columns={'Inspection Date Time':'InspectionDateTime','Job Serial Number': 'JobSerialNumber',
                         'Layout':'Layout','RUNOUT':'RUNOUT','DIA':'DIA','Shift':'Shift'},inplace=True)
    
    #Writing data to the sql table
    data.to_sql(name='CSVSQL2_TEST',con=engine,index=False,if_exists='append')
    print("Data Inserted Successfully!!!!")



#Calling the program

schedule.every().day.at("09:31").do(csv_to_sql)
# schedule.every().day.at("00:30").do(csv_to_sql)
# schedule.every().day.at("07:30").do(csv_to_sql)

while True:
    schedule.run_pending()
    time.sleep(1)